<a href="https://colab.research.google.com/github/plaban1981/Dphi_hackathon/blob/main/rank3_concrete_crack_classification_pytorch_lightining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#paths
folder_path = "/content/drive/MyDrive/Projects/Concrete crack detection/"
model_path = "/content/drive/MyDrive/Projects/Concrete crack detection/models/"
sub_path = "/content/drive/MyDrive/Projects/Concrete crack detection/subs/"
test_images = "/content/content/concrete_cracked_images/test"

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1ZKrbsUlrsTgJSlFMz2EGmTINFCQ8PcjO',
dest_path='content/concrete_crack_data.zip',
unzip=True)

Unzipping...Done.


In [ ]:
#for tpu
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev

#Creating Folds(Don't Run again)

In [ ]:
#creating train_dataset
import pandas as pd
import numpy as np
import os

folders = ['/content/content/concrete_cracked_images/train/Positive', '/content/content/concrete_cracked_images/train/Negative']
for fol in range(0,2):
  if fol == 0:
    pos_images = os.listdir(folders[fol])
    pos_labels = [1] * len(pos_images)
  else:
    neg_images = os.listdir(folders[fol])
    neg_labels = [0] * len(neg_images)

In [ ]:
print(f"Number of positive images: {len(pos_images)}")
print(f"Number of negative images: {len(neg_images)}")

Number of positive images: 14900
Number of negative images: 13818


In [ ]:
pos_image_paths = []
for img in pos_images:
  pos_image_paths.append(os.path.join(folders[0], img))

neg_image_paths = []
for img in neg_images:
  neg_image_paths.append(os.path.join(folders[1], img))

In [ ]:
pos_df = pd.DataFrame({"filename": pos_images,
                       "image_path": pos_image_paths,
                       "label": pos_labels})
neg_df = pd.DataFrame({"filename": neg_images,
                       "image_path": neg_image_paths,
                       "label": neg_labels})

In [ ]:
dfx = pd.concat([pos_df, neg_df], axis = 0).reset_index(drop = True).sample(frac = 1).reset_index(drop = True)

In [ ]:
#creating folds
from sklearn import model_selection
dfx["kfold"] = -1

dfx = dfx.sample(frac=1).reset_index(drop=True)

kf = model_selection.StratifiedKFold(n_splits=5)

for fold, (trn_, val_) in enumerate(kf.split(X=dfx, y=dfx.label.values)):
    print(len(trn_), len(val_))
    dfx.loc[val_, 'kfold'] = fold

22974 5744
22974 5744
22974 5744
22975 5743
22975 5743


In [ ]:
dfx.to_csv(folder_path + "Training_set.csv", index = False)

#Main Code

In [ ]:
!pip install pytorch-lightning
!pip install albumentations==0.4.6
!pip install efficientnet_pytorch
!pip install timm

In [ ]:
import os
import albumentations
from albumentations.pytorch import ToTensorV2
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.nn import functional as F
from torchvision import models
from torch.utils.data import TensorDataset, DataLoader, Dataset

from sklearn import metrics, model_selection, preprocessing

from PIL import Image
from PIL import ImageFilter

import cv2

from efficientnet_pytorch import EfficientNet

import timm

import pytorch_lightning as pl
from pytorch_lightning import metrics

from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

import warnings
warnings.filterwarnings('ignore')

In [ ]:
dfx = pd.read_csv("/content/drive/MyDrive/Projects/Concrete crack detection/Training_set.csv")
dfx_test = pd.read_csv("/content/content/concrete_cracked_images/Testing_set_concrete_crack.csv")

In [ ]:
test_image_path = []
for img in dfx_test['filename']:
  test_image_path.append(os.path.join(test_images, img))

In [ ]:
dfx_test['image_path'] = test_image_path
dfx_test['label'] = 0

In [ ]:
class ConcreteData(Dataset):
    def __init__(self, dataframe, transform, target):
        super().__init__()
        self.dataframe = dataframe
        self.transform = transform
        self.target = target
    
    def __len__(self):
        return self.dataframe.shape[0]
    
    def __getitem__(self, item):
      img_path = self.dataframe.iloc[item]['image_path']
      target = self.target[item]
      image = cv2.imread(img_path, 1)
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      if self.transform is not None:
        image = self.transform(image = image)['image']

      return image, torch.tensor(target)

In [ ]:
train_aug = albumentations.Compose([
            albumentations.CLAHE(p = 0.5),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ), ToTensorV2(),
            ])
  
        
valid_aug = albumentations.Compose([
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ), ToTensorV2(),
])

In [ ]:
class CNNClassifier(pl.LightningModule):
  def __init__(self, train_dataset, val_dataset, learning_rate = None, batch_size = 32):
    super().__init__()
    #model
    self.effnet = EfficientNet.from_pretrained('efficientnet-b4')
    self.classifier = nn.Sequential(nn.Dropout(0.1),
                                    nn.LeakyReLU(),
                                    nn.Linear(1000,2))
 
    self.learning_rate = learning_rate
    self.batch_size = batch_size
    self.save_hyperparameters()

    self.train_acc_metric = pl.metrics.classification.Accuracy()
    self.valid_acc_metric = pl.metrics.classification.Accuracy()

    self.train_dataset = train_dataset
    self.val_dataset = val_dataset

  def forward(self, x):
    x = self.effnet(x)
    x = self.classifier(x)
    return x

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
    schedular = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1
        )
    return [optimizer], [schedular]

  def training_step(self, batch, batch_idx):
    image, targets = batch
    y_pred = self.forward(image)
    loss = F.binary_cross_entropy_with_logits(y_pred, targets.type_as(y_pred))
    train_acc_batch = self.train_acc_metric(y_pred, targets)
    self.log('train_acc_batch', train_acc_batch, prog_bar = True)
    self.log('train_loss_batch', loss, prog_bar = True)
    return {
        'loss': loss,
        'y_pred': y_pred,
        'y_true': targets
    }

  def training_epoch_end(self, outputs):
      train_acc = self.train_acc_metric.compute()
      print(f"Training_ACC: {train_acc}")
      self.log('Training_acc_epoch', train_acc)

  def validation_step(self, batch, batch_idx):
      image, targets = batch
      y_pred = self.forward(image)
      loss = F.binary_cross_entropy_with_logits(y_pred, targets.type_as(y_pred))
      val_acc_batch = self.valid_acc_metric(y_pred, targets)
      self.log('val_acc_batch', val_acc_batch, prog_bar = True)
      self.log('val_loss_batch', loss, prog_bar = True)
      return {
          'loss': loss,
          'y_pred': y_pred,
          'target': targets
      }

  def validation_epoch_end(self, outputs):
        val_acc = self.valid_acc_metric.compute()
        print(f"Validation_ACC: {val_acc}")
        self.log('Validation_acc_epoch', val_acc)

  def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size = self.batch_size, shuffle = True)
      
  def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size = self.batch_size, shuffle = False)

In [ ]:
model_name = 'effnet_b4/'
def run(fold):
    check_path = model_path + model_name + str(fold)
    logging_path = '/content/drive/MyDrive/Projects/Concrete crack detection/logs/'
    df_train = dfx[dfx.kfold != fold].reset_index(drop=True)
    df_valid = dfx[dfx.kfold == fold].reset_index(drop=True)

    df_train = df_train.reset_index(drop=True)
    df_valid = df_valid.reset_index(drop=True)

    train_targets = F.one_hot(torch.tensor(df_train.label.values), num_classes=2)
    valid_targets = F.one_hot(torch.tensor(df_valid.label.values), num_classes=2)

    train_dataset = ConcreteData(
        dataframe = df_train,
        transform = train_aug,
        target = train_targets
    )

    valid_dataset = ConcreteData(
        dataframe = df_valid,
        transform = valid_aug,
        target = valid_targets
    )

    checkpointer = ModelCheckpoint(
    monitor = 'val_loss_batch',
    dirpath = check_path,
    filename = 'effnet-{epoch:02d}-{val_loss_batch:2f}',
    mode = 'min',
    save_top_k = 1)

    early_stopping = EarlyStopping(
    monitor = 'val_loss_batch',
    patience = 3,
    mode = 'min')

    learning_rate_monitor = LearningRateMonitor(
    logging_interval = 'epoch')

    callbacks = [checkpointer, early_stopping, learning_rate_monitor]
    logger = TensorBoardLogger(logging_path, name = model_name + f"fold_{fold}")
      
    model = CNNClassifier(train_dataset, valid_dataset, learning_rate=5e-5, batch_size=32)
    trainer = pl.Trainer(
    auto_lr_find = True,
    logger = logger,
    callbacks = callbacks,
    max_epochs = 50,
    progress_bar_refresh_rate = 20,
    gpus = 1,
    # tpu_cores = 8,
    # accumulate_grad_batches=2,
    precision = 16)


    trainer.fit(model)

In [ ]:
for fold in range(5):
    print(f"Running fold: {fold}")
    run(fold)

Running fold: 0


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


Loaded pretrained weights for efficientnet-b4



  | Name             | Type         | Params
--------------------------------------------------
0 | effnet           | EfficientNet | 19.3 M
1 | classifier       | Sequential   | 2.0 K 
2 | train_acc_metric | Accuracy     | 0     
3 | valid_acc_metric | Accuracy     | 0     
--------------------------------------------------
19.3 M    Trainable params
0         Non-trainable params
19.3 M    Total params


Validation_ACC: 0.5078125


...

Validation_ACC: 0.998955249786377
Training_ACC: 0.9991077184677124


Validation_ACC: 0.9992164373397827
Training_ACC: 0.9991077184677124


Validation_ACC: 0.9991293549537659
Training_ACC: 0.9993689060211182


Validation_ACC: 0.9992164373397827
Training_ACC: 0.9991948008537292


Validation_ACC: 0.9991293549537659
Training_ACC: 0.9995865225791931



#Predictions

In [ ]:
model_checks = "/content/drive/MyDrive/Projects/Concrete crack detection/models/effnet_b4/"
checkpoints = []
for i in range(5):
  p = f"{model_checks}{i}"
  checkpoints.append(os.listdir(p)[0])

In [ ]:
checkpoints

['effnet-epoch=05-val_loss_batch=0.000307.ckpt',
 'effnet-epoch=04-val_loss_batch=0.002214.ckpt',
 'effnet-epoch=02-val_loss_batch=0.002424.ckpt',
 'effnet-epoch=03-val_loss_batch=0.001476.ckpt',
 'effnet-epoch=04-val_loss_batch=0.004747.ckpt']

In [ ]:
class ConcreteTestData(Dataset):
    def __init__(self, dataframe, data_dir, transform, target):
        super().__init__()
        self.dataframe = dataframe
        self.data_dir = data_dir
        self.transform = transform
        self.target = target
    
    def __len__(self):
        return self.dataframe.shape[0]
    
    def __getitem__(self, item):
      img_id = self.dataframe.iloc[item]['filename']
      img_path = os.path.join(self.data_dir, img_id)
      target = self.target[item]
      image = cv2.imread(img_path, 1)
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      if self.transform is not None:
        image = self.transform(image = image)['image']

      return image, torch.tensor(target)

In [ ]:
test_path = "/content/content/concrete_cracked_images/test"
test_dataset = ConcreteTestData(
    dataframe = dfx_test,
    data_dir = test_path,
    transform = valid_aug,
    target = dfx_test.label.values
)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle = False)

In [ ]:
from tqdm import tqdm
def create_prediction(model_check_path):
  model = CNNClassifier.load_from_checkpoint(checkpoint_path=model_check_path).cuda()
  model.eval()
  final_preds = None
  for batch in tqdm(test_loader):
    images = batch[0].cuda()
    preds = torch.sigmoid(model(images)).cpu().detach().numpy()
    del(images)
    if final_preds is None:
      final_preds = preds
    else:
      final_preds = np.vstack((final_preds,preds))
  del(model)
  return final_preds

In [ ]:
fold_probs = []
for i in range(5):
  probs_name = f"effnet_b4_fold_{i}"
  p = f"{model_checks}{i}/{checkpoints[i]}"
  probs = create_prediction(p)
  fold_probs.append(probs)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b4-6ed6700e.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b4-6ed6700e.pth



Loaded pretrained weights for efficientnet-b4


  0%|          | 0/353 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b4


  0%|          | 0/353 [00:00<?, ?it/s]

...

  0%|          | 0/353 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b4


  0%|          | 0/353 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b4


100%|██████████| 353/353 [01:24<00:00,  4.16it/s]


In [ ]:
f_1 = fold_probs[0]
f_2 = fold_probs[1]
f_3 = fold_probs[2]
f_4 = fold_probs[3]
f_5 = fold_probs[4]

In [ ]:
f = (f_1 + f_2 + f_3 + f_4 + f_5)/5

In [ ]:
f = f.argmax(axis = 1)

In [ ]:
sub = pd.read_csv("/content/content/concrete_cracked_images/sample_submission.csv")

In [ ]:
labels = {0: "Negative",
          1: "Positive"}

In [ ]:
sub['prediction'] = f

In [ ]:
sub_1 = sub['prediction'].map(labels)
sub_1.to_csv("fold_1-3.csv", index = False)

In [ ]:
pd.Series(f).value_counts()

0    6180
1    5102
dtype: int64

In [ ]:
np.save("effnet_b4_fold_probs.npy",np.array(fold_probs))